# Import


In [1]:
#Dependencies
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
import tensorflow as tf
from torch.utils.data import DataLoader
import itertools
import torch

import os 

from collections import defaultdict
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchtools.optim import Ranger
from ranger import Ranger

import tqdm
import random
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity as cs
import seaborn as sns

# Data 
books_item5_user20.csv

In [2]:
datapath = os.getenv('HOME') + '/aiffel/Hackathon/books_item5_user20.csv'
columns =  ['UserID', 'BookID', 'Rating', 'Timestamp']
ratings_df = pd.read_csv(datapath, sep = ",", names = columns, dtype = int)
ratings_df.astype(int)
ratings_df.head()

,UserID,BookID,Rating,Timestamp
0,32,60751,5,1112140800
1,32,291860,5,1466380800
2,32,140336,5,1436400000
3,32,197628,5,1383436800
4,32,208357,5,1381190400


In [3]:
print(ratings_df.shape)

(13478380, 4)


In [4]:
ratings_df['UserID'].nunique()

395528

In [5]:
ratings_df['BookID'].nunique()

301387

In [6]:
# #Loading datasets
# ratings_list = [i.strip().split("::") for i in open('/content/gdrive/My Drive/RLProject/Data/ml-1m/ratings.dat', 'r').readlines()]
# users_list = [i.strip().split("::") for i in open('/content/gdrive/My Drive/RLProject/Data/ml-1m/users.dat', 'r').readlines()]
# movies_list = [i.strip().split("::") for i in open('/content/gdrive/My Drive/RLProject/Data/ml-1m/movies.dat',encoding='latin-1').readlines()]
# ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
# movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
# movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [7]:
#prepare_dataset
#using ratings_df
users_df = ratings_df.sort_values(["UserID","Timestamp"]).set_index("UserID").fillna(0).drop("Timestamp",axis=1)
users = dict(tuple(users_df.groupby("UserID")))


In [78]:
columns =  ['UserID', 'BookID', 'Rating', 'timestamp']
ratings_df = pd.read_csv(datapath, names = columns, dtype = int)

In [91]:
ratings_df1 = ratings_df.drop_duplicates('BookID', keep= 'first')

In [92]:
ratings_df1.shape

(301387, 4)

In [93]:
# ratings_df2 = ratings_df1.drop_duplicates('BookID', keep = 'last')
# print('1')

In [95]:

R_df = ratings_df1.pivot(index = 'UserID', columns ='BookID', values = 'Rating').fillna(0)
R_df = R_df.astype(int)

MemoryError: Unable to allocate 111. GiB for an array with shape (49608, 301387) and data type float64

In [72]:
R_df.head()

BookID,1,2,3,4,5
UserID,,,,,
0,0,0,0,1437696000,0
1,0,0,0,1415836800,0
2,0,0,0,0,1519948800
3,0,0,0,1377907200,0
4,0,0,0,0,1489536000


In [13]:
#might be used in the user dependednt state representation
userids = list(R_df.index.values) #list of userids
idx_to_userids = {i:userids[i] for i in range(len(userids))}
userids_to_idx = {userids[i]:i for i in range(len(userids))}

In [14]:
len(userids)

301387

In [15]:
#list of movie ids
columns = list(R_df)
idx_to_id = {i:columns[i] for i in range(len(columns))}
id_to_idx = {columns[i]:i for i in range(len(columns))}

In [16]:
#to get item embeddings
#R_df[userid][movieid]

## Getting Embeddings of User and Item(Movie Id's)


In [17]:
# R = R_df.values
# user_ratings_mean = np.mean(R, axis = 1)
# R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [97]:
R = ratings_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [99]:
R.shape

(13478380, 4)

In [98]:
#Movie Embeddings
U, sigma, Vt = svds(R_demeaned, k = 4)
# print(Vt.shape)
V = Vt.transpose()
# print(V.shape)
movie_list = V.tolist()
movie_embeddings_dict = {columns[i]:tf.convert_to_tensor(movie_list[i]) for i in range(len(columns))}

ValueError: k must be between 1 and min(A.shape), k=4

In [19]:
user_list = U.tolist()
user_embeddings_dict =  {userids[i]:tf.convert_to_tensor(user_list[i]) for i in range(len(userids))}

In [20]:
len(user_embeddings_dict)

301387

## Train and Test Dataset

In [21]:
#Getting users with positive rating count greater than 10
# 10 == 'N' positively interacted items
# from collections import defaultdict
# from collections import Counter
users_dict = defaultdict(dict)
users_id_list = set()
for user_id in users:
#   rating_freq = Counter(users[user_id]["Rating"].values)
#   if rating_freq['4']+rating_freq['5']<10 :
#     continue    
#   else:
    users_id_list.add(user_id)
    users_dict[user_id]["item"] = users[user_id]["BookID"].values
    users_dict[user_id]["rating"] = users[user_id]["Rating"].values
  


In [22]:
users_id_list = np.array(list(users_id_list))

In [23]:
len(users_id_list)

395528

In [24]:
#choosing default train_test_split of 25%
train_users,test_users = train_test_split(users_id_list)
print(train_users[:2])

[ 15962 126331]


In [25]:
print(test_users[:2])

[ 42146 191533]


In [26]:
from torch.utils.data import Dataset
class UserDataset(Dataset):
    def __init__(self,users_list,users_dict):
        self.users_list = users_list
        self.users_dict = users_dict

    def __len__(self):
        return len(self.users_list)

    def __getitem__(self,idx):
        user_id = self.users_list[idx]
        items = [('1',)]*10
        ratings = [('0',)]*10
        j=0
        for i,rate in enumerate(self.users_dict[user_id]["rating"]):
            if int(rate) >3 and j < 10:
                items[j] = self.users_dict[user_id]["item"][i]
                ratings[j] = self.users_dict[user_id]["rating"][i]
                j += 1
        # item = list(self.users_dict[user_id]["item"][:])
        # rating = list(self.users_dict[user_id]["rating"][:])
        size = len(items)
    
        return {'item':items,'rating':ratings,'size':size,'userid':user_id,'idx':idx}



In [27]:
train_users_dataset = UserDataset(train_users,users_dict)
test_users_dataset = UserDataset(test_users,users_dict)

In [28]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_users_dataset,batch_size=1)
test_dataloader = DataLoader(test_users_dataset,batch_size=1)

In [29]:
train_num = len(train_dataloader)
print(train_num)

296646


# State Representation Models

In [30]:
def drrave_state_rep(userid_b,items,memory,idx):
    user_num = idx
    H = [] #item embeddings
    user_n_items = items
    user_embeddings = torch.Tensor(np.array(user_embeddings_dict[userid_b[0]]),).unsqueeze(0)
    for i,item in enumerate(user_n_items):
        H.append(np.array(movie_embeddings_dict[item[0]]))
    avg_layer = nn.AvgPool1d(1)  # pooling layer 사용 
    item_embeddings = avg_layer(torch.Tensor(H,).unsqueeze(0)).permute(0,2,1).squeeze(0)
    state = torch.cat([user_embeddings,user_embeddings*item_embeddings.T,item_embeddings.T])
    return state #state tensor shape [21,100]



In [31]:
# def drru_state_rep(userid_b,items,memory,idx):
#     user_num = idx
#     H = []
#     user_n_items = items
#     user_embeddings = user_embeddings_dict[userid_b[0]]
#     for i,item in enumerate(user_n_items):
#         ui = np.array(user_embeddings) * np.array(movie_embeddings_dict[item[0]])
#         H.append(ui)

#     pairs = list(itertools.combinations(memory[user_num], 2))
#     for item1,item2 in pairs:
#         pair1 =  np.array(movie_embeddings_dict[str(int(item1))])
#         pair2 = np.array(movie_embeddings_dict[str(int(item2))])

#         product = pair1*pair2
#         H.append(product)
#     state = torch.Tensor(H,)
#     return state #state tensor shape [55,100]


In [32]:
# def drrp_state_rep(items,memory,idx):
#   user_num = idx
#   H = []
#   user_n_items = items
#   for i,item in enumerate(user_n_items):
#     H.append(np.array(movie_embeddings_dict[item[0]]))
  
#   pairs = list(itertools.combinations(memory[user_num], 2))
#   for item1,item2 in pairs:
#     pair1 =  np.array(movie_embeddings_dict[str(int(item1))])
#     pair2 = np.array(movie_embeddings_dict[str(int(item2))])
#     product = pair1*pair2
#     H.append(product)
#   state = torch.Tensor(H,)
#   return state

In [33]:
# just n items and their embeddings used to represent state
def state_rep(item_b):
    state = []
    user_embeddings = np.zeros((len(columns),100))
    movie_ids = list(item[0] for item in item_b)
    for i,subitem in enumerate(user_embeddings):
        if idx_to_id[i] in movie_ids:
            user_embeddings[i] = np.array(movie_embeddings_dict[idx_to_id[i]])
        else:
            user_embeddings[i] = np.zeros((100,))
    state = torch.Tensor(user_embeddings,)
    return torch.reshape(state,[-1]) 

# Actor, Critic Module

In [34]:
!pip install git+https://github.com/pabloppp/pytorch-tools@0.2.4 -U

  Cloning https://github.com/pabloppp/pytorch-tools (to revision 0.2.4) to /tmp/pip-req-build-9js3mimi
  Running command git clone -q https://github.com/pabloppp/pytorch-tools /tmp/pip-req-build-9js3mimi
  Running command git checkout -q 86c73996537002ab29e7e40f925cb90756f58156
  Created wheel for torchtools: filename=torchtools-0.2.4-py3-none-any.whl size=19202 sha256=357da95f8ae76bf9c1dc5ee978da2daa48f1dc67f62e2b9a2223bf762d79f877
  Stored in directory: /tmp/pip-ephem-wheel-cache-dx4oe946/wheels/e9/db/08/ceb66f88d5c31d39af6901601b4b9084b365c3b484105ee2f5
Successfully built torchtools
  Attempting uninstall: torchtools
    Found existing installation: torchtools 0.2.4
    Uninstalling torchtools-0.2.4:
      Successfully uninstalled torchtools-0.2.4


In [35]:
#Dependencies

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchtools.optim import Ranger
# import tqdm
# import random
# import matplotlib.pyplot as plt
#uncomment to use adam
# from torch.optim import Adam



In [36]:
#Actor Model:
#Generating an action a based on state s

class Actor(torch.nn.Module):
    def __init__(self, input_dim, output_dim,hidden_dim):
        super(Actor, self).__init__()

        self.drop_layer = nn.Dropout(p=0.5)        
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, output_dim)
  

    def forward(self, state):
        x = F.relu(self.linear1(state))
        # print(x.shape)
        x = self.drop_layer(x)
        x = F.relu(self.linear2(x))
        # print(x.shape)
        x = self.drop_layer(x)
        # x = torch.tanh(self.linear3(x)) # in case embeds are -1 1 normalized
        x = self.linear3(x) # in case embeds are standard scaled / wiped using PCA whitening
        # return state, x
        return x # state = self.state_rep(state)
   
      




In [37]:
class Critic(nn.Module):
    def __init__(self,input_dim,output_dim,hidden_dim):

        super(Critic, self).__init__()
        
        self.drop_layer = nn.Dropout(p=0.5)
    
        self.linear1 = nn.Linear(input_dim + output_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, 1)

    def forward(self,state,action):    
        x = torch.cat([state, action], 1)
        # print(x.shape)
        x = F.relu(self.linear1(x))
        x = self.drop_layer(x)
        x = F.relu(self.linear2(x))
        x = self.drop_layer(x)
        x = self.linear3(x)
        return x



In [38]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        # print(batch)
        state, action, reward, next_state = map(np.stack,zip(*batch))
        return state, action, reward, next_state
    
    def __len__(self):
        return len(self.buffer)

In [39]:
device = 'cpu'
# cuda = torch.device('cuda')

In [40]:
#used for plotting purposes
p_loss = []
v_loss = []

In [41]:
def ddpg_update(batch_size=1, 
                gamma = 0.6,
                min_value=-np.inf,
                max_value=np.inf,
                soft_tau=1e-2):
    
    state, action, reward, next_state = replay_buffer.sample(batch_size)
    state      = torch.FloatTensor(state).to(device)

    next_state = torch.FloatTensor(next_state).to(device)

    action     = torch.FloatTensor(action).to(device)
    reward     = torch.FloatTensor(reward).to(device)
    # print(state.shape)
    # print(policy_net(state).shape)
    policy_loss = value_net(state, policy_net(state))
    policy_loss = -policy_loss.mean()
    p_loss.append(policy_loss)
    next_action    = target_policy_net(next_state)
    target_value   = target_value_net(next_state, next_action.detach())
    expected_value = reward + gamma * target_value
    expected_value = torch.clamp(expected_value, min_value, max_value)

    value = value_net(state, action)
    # print("1")
    value_loss = value_criterion(value, expected_value.detach())
    # print("2")
    v_loss.append(value_loss)
    policy_optimizer.zero_grad()
    # print("3")
    policy_loss.backward()
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
                target_param.data.copy_(
                    target_param.data * (1.0 - soft_tau) + param.data * soft_tau
                )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

In [42]:
#initializing actor and critic networks for drru and drrp state representation

value_net = Critic(5500,100,256)
policy_net = Actor(5500,100,256)

target_value_net = Critic(5500,100,256)
target_policy_net = Actor(5500,100,256)


target_policy_net.eval()
target_value_net.eval()

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_criterion = nn.MSELoss()
value_optimizer      = Ranger(value_net.parameters(),  lr=1e-4)
policy_optimizer     = Ranger(policy_net.parameters(), lr=1e-4)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [43]:
#initializing for drrave state representation

value_net = Critic(2100,100,256)
policy_net = Actor(2100,100,256)

target_value_net = Critic(2100,100,256)
target_policy_net = Actor(2100,100,256)


target_policy_net.eval()
target_value_net.eval()

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_criterion = nn.MSELoss()
value_optimizer      = Ranger(value_net.parameters(),  lr=1e-4)
policy_optimizer     = Ranger(policy_net.parameters(), lr=1e-4)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [44]:
print(policy_net)

Actor(
  (drop_layer): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=2100, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
)


In [45]:
replay_buffer_size = 10000
replay_buffer = ReplayBuffer(replay_buffer_size)

memory = np.ones((train_num,10))*-1

In [46]:
def get_action(state,action_emb,userid_b,item_b,preds):
    action_emb = torch.reshape(action_emb,[1,100]).unsqueeze(0)
    item_embedding = []
    for movie in users_dict[userid_b[0]]["item"]:  
        item_embedding.append(np.array(movie_embeddings_dict[movie]))
    item_embedding = torch.Tensor(item_embedding,)
    items = item_embedding.T.unsqueeze(0)
    m = torch.bmm(action_emb,items).squeeze(0)  #torch.bmm : batch 행렬 곱연산
    sorted_m,indices = torch.sort(m,descending=True)
    index_list = list(indices[0])
    for i in index_list:
        if users_dict[userid_b[0]]["item"][i] not in preds:    
            preds.add(users_dict[userid_b[0]]["item"][i])
            return int(i)
 

In [47]:
def update_memory(memory,action,idx):
    memory[idx] = list(memory[idx,1:])+[action]

In [48]:
rate = 0

#Training

In [49]:
preddict = dict()
it = iter(train_dataloader)
for episode in tqdm.tqdm(range(train_num-1)):    
    batch_size= 1
    preds = set()
    first = next(it)
    item_b,rating_b,size_b,userid_b,idx_b = first['item'],first['rating'],first['size'],first['userid'],first['idx']
    memory[idx_b] = [item[0] for item in item_b]
    state = drrave_state_rep(userid_b,item_b,memory,idx_b)
    for j in range(5):    
        state_rep =  torch.reshape(state,[-1])
        action_emb = policy_net(state_rep)
        action = get_action(state,action_emb,userid_b,item_b,preds)
        rate = int(users_dict[userid_b[0]]["rating"][action])
        try:
            ratings = (int(rate)-3)/2
        except:
            ratings = 0
        reward = torch.Tensor((ratings,))

        if reward > 0:
            update_memory(memory,int(users_dict[userid_b[0]]["item"][action]),idx_b)

        next_state = drrave_state_rep(userid_b,item_b,memory,idx_b)
        next_state_rep = torch.reshape(next_state,[-1])
        replay_buffer.push(state_rep.detach().cpu().numpy(), action_emb.detach().cpu().numpy(), reward, next_state_rep.detach().cpu().numpy())
        if len(replay_buffer) > batch_size:
            ddpg_update()

        state = next_state
    preddict[userid_b[0]] = preds


  0%|          | 0/296645 [00:00<?, ?it/s]


KeyError: tensor(15962)

In [ ]:
plt.plot(v_loss)

In [ ]:
plt.plot(p_loss)

#Testing

In [ ]:
#Getting Cosine similarity of recommended items for a particular userid that has been evaluated
# from sklearn.metrics.pairwise import cosine_similarity as cs
# import seaborn as sns

def get_cosine_sim(userid):
  test_pred = test_pred_dict[userid]
  for i,item in enumerate(users_dict[userid]["item"]):
    if item in test_pred:
      print(item,":",users_dict[userid]["rating"][i])

  test_embed = []
  for item in test_pred:
    test_embed.append(np.array(movie_embeddings_dict[item]))

  test_embed_array = np.array(test_embed)

  return test_embed_array

test_embed_array = get_cosine_sim(userid_b[0])
ax = sns.heatmap(cs(test_embed_array), linewidth=0.5)
plt.show()


In [ ]:
#prediction algorithm
it2 = iter(test_dataloader)
precision = 0
test_pred_dict = dict()
for j in range(len(test_dataloader)-1):  #session 돌리기 : timestamps 내에서 items들 
  first = next(it2)
  item_b,rating_b,size_b,userid_b,idx_b = first['item'],first['rating'],first['size'],first['userid'],first['idx']
  memory[idx_b] = [item[0] for item in item_b]
  state = drrave_state_rep(userid_b,item_b,memory,idx_b)
  count = 0
  test_pred = set()
  for j in range(5):  #policy network5번 돌리기 , 추천 5번 하기
    state_rep =  torch.reshape(state,[-1])
    action_emb = policy_net(state_rep)   # policy_net = actor : items들의 선호도 (rating)
    action = get_action(state,action_emb,userid_b,item_b,test_pred)
    rate = int(users_dict[userid_b[0]]["rating"][action])
    try:
      rating = (int(rate)-3)/2
    except:
      rating = 0
    reward = torch.Tensor((rating,))

    if reward > 0:
      count += 1
      update_memory(memory,int(users_dict[userid_b[0]]["item"][action]),idx_b)
    next_state = drrave_state_rep(userid_b,item_b,memory,idx_b)
    state = next_state
  precision += count/5
  test_pred_dict[userid_b[0]] = test_pred
print("p",precision/(len(test_dataloader)-1))

#Saving and Loading Models

In [ ]:
PATH = '/content/gdrive/My Drive/RLProject/Models/drravepolicy_net.pth'
torch.save(policy_net.state_dict(), PATH)

In [ ]:
value_PATH = '/content/gdrive/My Drive/RLProject/Models/drravevalue_net.pth'
torch.save(value_net.state_dict(), value_PATH)

tpolicy_PATH = '/content/gdrive/My Drive/RLProject/Models/drravetpolicy_net.pth'
torch.save(target_policy_net.state_dict(), tpolicy_PATH)

tvalue_PATH = '/content/gdrive/My Drive/RLProject/Models/drravetvalue_net.pth'
torch.save(target_value_net.state_dict(), tvalue_PATH)


In [ ]:
np.save('/content/gdrive/My Drive/RLProject/Models/train_dataloader',train_dataloader)
np.save('/content/gdrive/My Drive/RLProject/Models/test_dataloader',test_dataloader)

In [ ]:
policy_net = Actor(5500,100,256)
policy_net.load_state_dict(torch.load(PATH))
policy_net.eval()

value_net = Critic(5500,100,256)
value_net.load_state_dict(torch.load(value_PATH))
value_net.eval()

target_policy_net = Actor(5500,100,256)
target_policy_net.load_state_dict(torch.load(tpolicy_PATH))
target_policy_net.eval()

target_value_net = Critic(5500,100,256)
target_value_net.load_state_dict(torch.load(tvalue_PATH))
target_value_net.eval()


In [ ]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

train_data = np.load('/content/gdrive/My Drive/RLProject/Models/train_users.npy')

In [ ]:
np.load = np_load_old